In [0]:
#create schema and read df with our shcema
from pyspark.sql.types import *
customSchema = StructType([
                            StructField("Country", StringType(), True),
                            StructField("Gender", StringType(), True),
                            StructField("Employee_id", IntegerType(), True),
                            StructField("First_Name", StringType(), True),
                            StructField("Salary", IntegerType(), True),
                            StructField("Department_id", IntegerType(), True)
    
                        ])

# COMMAND ----------
df = spark.read.format("csv")\
.options(header='true',delimiter=',')\
.schema(customSchema)\
.load("/mnt/files/SampleEmployee.csv")


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

windowFuncCoun1 = Window.partitionBy("Country").orderBy(desc("Salary"))
windowFuncCounMin = Window.partitionBy("Country").orderBy(asc("Salary"))

df \
    .withColumn("MaxSalaryPerCountry", max("Salary").over(windowFuncCoun1)) \
    .withColumn("MinSalaryPerCountry", min("Salary").over(windowFuncCounMin)) \
    .withColumn("RowNumber", row_number().over(windowFuncCoun1)) \
    .where(col("RowNumber") == 3) \
    .show()


+-------+------+-----------+----------+------+-------------+-------------------+-------------------+---------+
Country|Gender|Employee_id|First_Name|Salary|Department_id|MaxSalaryPerCountry|MinSalaryPerCountry|RowNumber|
+-------+------+-----------+----------+------+-------------+-------------------+-------------------+---------+
 Canada|Female| 3| Kellina| 96879| 12| 106837| 62228| 3|
 USA| Male| 11| Ugo|108740| 9| 121493| 67525| 3|
+-------+------+-----------+----------+------+-------------+-------------------+-------------------+---------+

In [0]:
#create window function to flag female and male, rank with a gap and dense_rank without
windowFunctionCoun2 = Window.partitionBy("Country").orderBy("Gender")

df \
    .withColumn("Rank",rank().over(windowFunctionCoun2)) \
    .withColumn("Dense_Rank",dense_rank().over(windowFunctionCoun2))\
    .show()

+-------+------+-----------+----------+------+-------------+----+----------+
Country|Gender|Employee_id|First_Name|Salary|Department_id|Rank|Dense_Rank|
+-------+------+-----------+----------+------+-------------+----+----------+
 Canada|Female| 2|Kristopher| 69531| 10| 1| 1|
 Canada|Female| 3| Kellina| 96879| 12| 1| 1|
 Canada|Female| 4| Tobit| 76070| 4| 1| 1|
 Canada| Male| 1| Auberon|105623| 4| 4| 2|
 Canada| Male| 5| Missy| 62228| 7| 4| 2|
 Canada| Male| 6| Faustina|106837| 5| 4| 2|
 Canada| Male| 7| Alic| 77729| 17| 4| 2|
 USA|Female| 12| Kathy|121493| 8| 1| 1|
 USA|Female| 14| Loni|119141| 8| 1| 1|
 USA| Male| 8| Gabey| 81590| 7| 3| 2|
 USA| Male| 9| Hammad| 71206| 13| 3| 2|
 USA| Male| 10| Kanya| 98642| 11| 3| 2|
 USA| Male| 11| Ugo|108740| 9| 3| 2|
 USA| Male| 13| Fannie| 67525| 8| 3| 2|
+-------+------+-----------+----------+------+-------------+----+----------+

In [0]:
#create a window function and add in to df that for each gender in different country, display the highest salary
windowFuncCoun3 = Window.partitionBy("Country","Gender").orderBy(desc("Salary"))

df\
    .withColumn("MaxSalaryPerCountryPerGender", max("Salary").over(windowFuncCoun3)) \
    .show()

+-------+------+-----------+----------+------+-------------+----------------------------+
Country|Gender|Employee_id|First_Name|Salary|Department_id|MaxSalaryPerCountryPerGender|
+-------+------+-----------+----------+------+-------------+----------------------------+
 Canada|Female| 3| Kellina| 96879| 12| 96879|
 Canada|Female| 4| Tobit| 76070| 4| 96879|
 Canada|Female| 2|Kristopher| 69531| 10| 96879|
 Canada| Male| 6| Faustina|106837| 5| 106837|
 Canada| Male| 1| Auberon|105623| 4| 106837|
 Canada| Male| 7| Alic| 77729| 17| 106837|
 Canada| Male| 5| Missy| 62228| 7| 106837|
 USA|Female| 12| Kathy|121493| 8| 121493|
 USA|Female| 14| Loni|119141| 8| 121493|
 USA| Male| 11| Ugo|108740| 9| 108740|
 USA| Male| 10| Kanya| 98642| 11| 108740|
 USA| Male| 8| Gabey| 81590| 7| 108740|
 USA| Male| 9| Hammad| 71206| 13| 108740|
 USA| Male| 13| Fannie| 67525| 8| 108740|
+-------+------+-----------+----------+------+-------------+----------------------------+